In [ ]:
%load_ext rpy2.ipython
from rpy2.robjects import r, pandas2ri
pandas2ri.activate()

In [ ]:
import math
import torch
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from transformers import BertTokenizerFast, BertModel, BertForMaskedLM
import numpy as np
import pandas as pd
import pickle
from collections import defaultdict
import re, os
import bisect
import kenlm
import mosestokenizer

In [ ]:
%%R
install.packages('lme4')
install.packages('MuMIn')
install.packages('lmerTest')
install.packages('ggplot2')
install.packages('infotheo')

## Preprocessing

### LM Scoring

In [ ]:
STRIDE = 200
def score_gpt(sentence):
      with torch.no_grad():
        all_log_probs = torch.tensor([], device=model.device)
        offset_mapping = []
        start_ind = 0

        while True:
            encodings = tokenizer(sentence[start_ind:], max_length=1022, truncation=True, return_offsets_mapping=True)
            tensor_input = torch.tensor([[tokenizer.bos_token_id] + encodings['input_ids'] + [tokenizer.eos_token_id]], device=model.device)
            output = model(tensor_input, labels=tensor_input)
            shift_logits = output['logits'][..., :-1, :].contiguous()
            shift_labels = tensor_input[..., 1:].contiguous()
            log_probs = torch.nn.functional.cross_entropy(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1), reduction='none')
            assert torch.isclose(torch.exp(sum(log_probs)/len(log_probs)),torch.exp(output['loss']))
            offset = 0 if start_ind == 0 else STRIDE-1
            all_log_probs = torch.cat([all_log_probs,log_probs[offset:-1]])
            offset_mapping.extend([(i+start_ind, j+start_ind) for i,j in encodings['offset_mapping'][offset:]])
            if encodings['offset_mapping'][-1][1] + start_ind == len(sentence):
                break
            start_ind += encodings['offset_mapping'][-STRIDE][1]
        return np.asarray(all_log_probs.cpu()), offset_mapping


def score_bert(sentence):
    mask_id = tokenizer.convert_tokens_to_ids('[MASK]')
    with torch.no_grad():
        all_log_probs = []
        offset_mapping = []
        start_ind = 0
        while True:
            encodings = tokenizer(sentence[start_ind:], max_length=512, truncation=True, return_offsets_mapping=True)
            tensor_input = torch.tensor([encodings['input_ids']], device=model.device)
            mask_input = tensor_input.clone()
            offset = 1 if start_ind == 0 else STRIDE
            for i, word in enumerate(encodings['input_ids'][:-1]):
                if i < offset:
                    continue
                mask_input[:,i]=mask_id
                output = model(mask_input, labels=tensor_input)
                log_probs = torch.nn.functional.log_softmax(output['logits'][:,i], dim=-1).squeeze(0)
                all_log_probs.append(-log_probs[tensor_input[0,i]].item())
                mask_input[:,i] = word
            
            offset_mapping.extend([(i+start_ind, j+start_ind) for i,j in encodings['offset_mapping'][offset:-1]])
            if encodings['offset_mapping'][-2][1] + start_ind >= (len(sentence)-1):
                break
            start_ind += encodings['offset_mapping'][-STRIDE-1][1]
            
        return all_log_probs, offset_mapping


MOSESTOKENIZER = mosestokenizer.MosesTokenizer("en")
MOSESDETOKENIZER = mosestokenizer.MosesDetokenizer("en")
def score_ngram(sentence):
    #return strange characters back to original form
    tokens = [MOSESDETOKENIZER([t]) for t in MOSESTOKENIZER(sentence)]
    tokenized_sentence = " ".join(tokens)
    spans = []
    word_start = 0
    for t in tokens:
        while sentence[word_start] != t[0]:
            word_start += 1
        spans.append((word_start, word_start+len(t)))
        word_start += len(t)
    scores = model.full_scores(tokenized_sentence, eos=False, bos=True)
    return np.array([-s[0] for s in scores]), spans
    


In [ ]:
MODEL = "gpt"

if MODEL == "bert":
    model = BertForMaskedLM.from_pretrained('bert-base-uncased')
    model.eval()
    if torch.cuda.is_available():
        model = model.cuda()
    tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
    score = score_bert
    
elif MODEL == "ngram":
    model = kenlm.Model('wiki.arpa')
    score = score_ngram
    
else:
    model = GPT2LMHeadModel.from_pretrained('gpt2')
    model.eval()
    if torch.cuda.is_available():
        model = model.cuda()
    tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')
    score = score_gpt
    

In [ ]:
# Sanity check for above function
a=['there is a book on the desk',
                'there is a plane on the desk',
                        'there is a books in the desk']
scores = [score(i) for i in a]
print([sum(s[0]) for s in scores])

In [ ]:
import nltk
nltk.download('punkt')
def string_join(x, j=''):
    return j.join(x)

def get_word_mapping(words):
    offsets = []
    pos = 0
    for w in words:
        offsets.append((pos,pos+len(w)))
        pos += len(w) + 1
    return offsets

def string_to_log_probs(string, probs, offsets):
    words = string.split()
    agg_log_probs = []
    word_mapping = get_word_mapping(words)
    cur_prob = 0
    cur_word_ind = 0
    for lp, ind in zip(probs, offsets):
        cur_prob += lp
        start, end = ind
        start_cur_word, end_cur_word = word_mapping[cur_word_ind]
        if end == end_cur_word:
            agg_log_probs.append(cur_prob)
            cur_prob = 0
            cur_word_ind += 1
    return agg_log_probs


### Unigram model

In [ ]:
def get_ngrams(sentence, n=3):
    words = sentence.split()
    words = ["BOS"]*(n-1) + words + ["EOS"]
    ngrams = []
    for i in range(len(words)-n+1):
        ngrams.append((tuple(words[i:i+n-1]), words[i+n-1]))
    return ngrams

def normalize(root, log=True):
    for prefix in root.keys():
        counts = root[prefix]
        total_counts = np.sum([counts[word] for word in counts.keys()])
        if log:
            root[prefix] = {k: np.log(v)- np.log(total_counts) for (k,v) in counts.items()}
        else:
            root[prefix] = {k: v/total_counts  for (k,v) in counts.items()}
    return root

def sampling_format(root, normalize=True):
    for prefix in root.keys():
        v = root[prefix]
        words = list(v.keys())
        counts = np.array([v[word] for word in words])
        if normalize:
            counts = counts/np.sum(counts)
        root[prefix] = (words, np.log(counts))
    return root

def create_ngram_model(filename, n, outfile):
    with open(filename, 'r') as f:
        root = defaultdict(lambda: defaultdict(int))
        for sentence in f:
            if not sentence:
                continue
            ngrams = get_ngrams(sentence.lower().strip(), n)
            for ngram in ngrams:
                root[ngram[0]][ngram[1]] += 1
        root = normalize(root, log=False)
        pickle.dump(dict(root), open(outfile, "wb"))
        return root

def get_corpus_mean(filename, n=100000):
    with open(filename, 'r') as f:
        probs = []
        for i, sentence in enumerate(f):
            if i == n:
                break
            if sentence.isspace():
                continue
            probs.extend(score(sentence.strip())[0])
        return np.mean(probs)
n = 1
root = create_ngram_model("wikitext-103/wiki.train.tokens", 1, "unigram.pkl")
#root = pickle.load(open("unigram.pkl", "rb"))
CORPUS_MEAN = 3.88445 #get_corpus_mean("wikitext-103/wiki.train.tokens")

### Corpus Statistics

In [ ]:
from scipy.special import log_softmax, softmax
POWER_RANGE = np.arange(0.25, 4, 0.25)
POWER_RANGE2 = np.arange(1, 3, 0.25)
def power(x, y): 
    return np.mean(x**y)

def ent(x):
    l_soft = log_softmax(-x)
    return -sum(np.exp(l_soft)*l_soft)

def ent2(x):
    return sum(np.exp(-x)*x)

def r_ent(x, k=2):
    soft = softmax(-x)
    return 1/(1-k)*np.log(sum(soft**k))

def r_ent2(x, k=2):
    return 1/(1-k)*np.log(sum(np.exp(-x)**k))

def local_diff(x):
    d = 0
    for i in range(len(x)-1):
        d += abs(x[i+1]-x[i])
    return d/len(x)

def local_diff2(x):
    d = 0
    for i in range(len(x)-1):
        d += (x[i+1]-x[i])**2
    return d/len(x)

def tokenize_to_sents(s):
    sents = []
    for sen in nltk.sent_tokenize(s):
        #failure case of sentence tokenizer
        if sen.split()[0] != "',":
            sents.append(sen)
        else:
            sents[-1] += sen
    return sents

def corpus_stats(df, probs_file='', split_sens=True, wiki_mean=True, save=False):
    stats = defaultdict(lambda: defaultdict(list))
    try:
        stats['log_probs'] = pickle.load(open(probs_file, "rb"))
    except:
        import itertools
        df, df_copy = itertools.tee(df)
        stats['log_probs'] = {i: score(s.strip()) for i,s in df_copy}
    lang_mean = CORPUS_MEAN if wiki_mean else np.mean(np.concatenate([s[0] for s in stats['log_probs'].values()])) 
    print("Using language mean surprisal:", lang_mean)
    for i, s in df:
      # remove leading and trailing white space
        s = s.strip()
        stats['split_string'][i] = s.split()
        if split_sens:
            lens = [len(sen.split()) for sen in tokenize_to_sents(s)]
            assert len(s.split()) == sum(lens)
            stats['sent_markers'][i] = np.cumsum(lens)
            stats['agg_log_probs'][i] = np.array(string_to_log_probs(s, *stats['log_probs'][i]))
            assert len(stats['agg_log_probs'][i]) == len(stats['split_string'][i])
        else:
            stats['agg_log_probs'][i] = stats['log_probs'][i][0]
            stats['sent_markers'][i] = [len(stats['agg_log_probs'][i])] 
        
        for j in range(len(stats['sent_markers'][i])):
            prev = 0 if not j else stats['sent_markers'][i][j-1]
            end = stats['sent_markers'][i][j]
            stats['log_prob_variance'][i].append(np.var(stats['agg_log_probs'][i][prev:end]))
            stats['log_prob_variance_lang'][i].append(np.mean((stats['agg_log_probs'][i][prev:end] - lang_mean)**2))
            stats['log_prob_max'][i].append(np.amax(stats['agg_log_probs'][i][prev:end]))
            stats['log_prob_mean'][i].append(np.mean(stats['agg_log_probs'][i][prev:end]))
            stats['log_prob_ldiff'][i].append(local_diff(stats['agg_log_probs'][i][prev:end]))
            stats['log_prob_ldiff2'][i].append(local_diff2(stats['agg_log_probs'][i][prev:end]))
            stats['len'][i].append(end-prev)
            
            for p in POWER_RANGE:
                if p < 1:
                    func1 = lambda x: r_ent(x, p) 
                    func2 = lambda x: r_ent2(x, p)
                elif p == 1:
                    func1 = lambda x: 1/ent(x) if ent(x) else 0
                    func2 = lambda x: 1/ent2(x) if ent2(x) else 0
                else:
                    func1 = lambda x: 1/r_ent(x, p) if r_ent(x, p) else 0
                    func2 = lambda x: 1/r_ent2(x, p) if r_ent2(x, p) else 0
                stats['log_prob_entropy_' + str(p)][i].append(func1(stats['agg_log_probs'][i][prev:end]))
                stats['log_prob_entropy_n_' + str(p)][i].append(func2(stats['agg_log_probs'][i][prev:end]))
                stats['log_prob_power_' + str(p)][i].append(power(stats['agg_log_probs'][i][prev:end], p))  
    if save: 
        pickle.dump(stats['log_probs'], open(probs_file, "wb"))
    return stats


In [ ]:
def add_aggregate_cols(aggregate_per_sentence, stats):
    def add_attribute(name):
        aggregate_per_sentence[name] = aggregate_per_sentence.apply(lambda x: stats[name][x['text_id_']][int(x['sentence_num_'])], axis=1)
    
    attributes = ['log_prob_mean','log_prob_max','log_prob_variance', 
                  'log_prob_variance_lang', 'log_prob_ldiff', 'log_prob_ldiff2', 'len']
    for i in POWER_RANGE:
        attributes.extend(['log_prob_entropy_'+str(i), 'log_prob_entropy_n_'+str(i), 'log_prob_power_'+str(i)])
    for a in attributes:
        add_attribute(a)
    aggregate_per_sentence['log_prob_std'] = np.sqrt(aggregate_per_sentence['log_prob_variance'])
    

def produce_aggregate_per_sentence(main_df, stats, line_col=None):
    aggregate_per_sentence = main_df.groupby(by=["WorkerId","text_id", "sentence_num"]).agg({"time":[np.sum, np.mean, np.count_nonzero], 
                                                                                                               "word_len":[np.sum, np.mean], 
                                                                                                               "freq":[np.sum, np.mean]}).reset_index()
    aggregate_per_sentence.columns = ['_'.join(col).strip() for col in aggregate_per_sentence.columns.values]
    if line_col:
        aggregate_per_sentence['line_breaks'] = main_df.groupby(by=["WorkerId","text_id", "sentence_num"], as_index = False).agg({line_col: lambda x:len(np.unique(x))})[line_col]
    add_aggregate_cols(aggregate_per_sentence, stats)
    return aggregate_per_sentence
    

In [ ]:
def add_df_columns(df, stats, rolling_vals=False):
    df['centered_time'] = df['time'] - df.groupby(by=["WorkerId"]).transform('mean')["time"]
    df['prev_word'] = df.apply(lambda x: stats['split_string'][x['text_id']][x['new_ind']-1] if x['new_ind']-1 >= 0 else '', axis=1)
    df['log_prob'] = df.apply(lambda x: stats['agg_log_probs'][x['text_id']][x['new_ind']], axis=1)
    df['prev_log_prob'] = df.apply(lambda x: stats['agg_log_probs'][x['text_id']][x['new_ind']-1] if x['new_ind']-1 >= 0 else 0, axis=1)
    df['prev2_log_prob'] = df.apply(lambda x: stats['agg_log_probs'][x['text_id']][x['new_ind']-2] if x['new_ind']-2 >= 0 else 0, axis=1)
    df['prev3_log_prob'] = df.apply(lambda x: stats['agg_log_probs'][x['text_id']][x['new_ind']-3] if x['new_ind']-3 >= 0 else 0, axis=1)
    df['word_len'] = df.apply(lambda x: len(x['word']), axis=1)
    df['prev_word_len'] = df.apply(lambda x: len(x['prev_word']), axis=1)
    df['freq'] = df.apply(lambda x: root[()].get(x['word'],0), axis=1)
    df['prev_freq'] = df.apply(lambda x: root[()].get(x['prev_word'],0), axis=1)


In [ ]:
def ordered_string_join(x, j=''):
    s = sorted(x, key=lambda x: x[0])
    a,b = list(zip(*s))
    return a, j.join(b)

## Datasets

### Natural Stories

In [ ]:
gpt3_probs = pd.read_csv("https://raw.githubusercontent.com/languageMIT/naturalstories/master/probs/all_stories_gpt3.csv")
# To get same indexing as stories db
gpt3_probs["story"] = gpt3_probs["story"] + 1
gpt3_probs['len'] = gpt3_probs.groupby("story", sort=False)['offset'].shift(periods=-1, fill_value=0) - gpt3_probs['offset'] 
gpt3_probs['new_token'] = gpt3_probs.apply(lambda x: x['token'] if x['len'] == len(x['token']) else x['token'] + ' ', axis=1) 

In [ ]:
stories_df = gpt3_probs.groupby(by=["story"], sort=False).agg({"new_token":[string_join]}).reset_index()
ns_stats = corpus_stats(zip(stories_df['story'], stories_df['new_token', 'string_join']), "ns_gpt_probs.pkl")

In [ ]:
reading_times_df = pd.read_csv("https://raw.githubusercontent.com/languageMIT/naturalstories/master/naturalstories_RTS/processed_RTs.tsv", sep='\t').drop_duplicates()
reading_times_df.rename(columns = {'RT':'time', 
                                   'item': 'text_id'}, inplace = True)
reading_times_df['new_ind'] = reading_times_df['zone'] - 1
reading_times_df['sentence_num'] = reading_times_df.apply(lambda x: bisect.bisect(ns_stats['sent_markers'][x['text_id']], x['new_ind']), axis=1)
# ref token is sanity check. should be same as word
reading_times_df['ref_token'] = reading_times_df.apply(lambda x: ns_stats['split_string'][x['text_id']][x['new_ind']], axis=1)
add_df_columns(reading_times_df, ns_stats)

In [ ]:
# looks like there's a small mispelling somewhere ;)
reading_times_df[reading_times_df['word'] != reading_times_df['ref_token']]

In [ ]:
ns_agg_per_sentence = produce_aggregate_per_sentence(reading_times_df, ns_stats)

### Provo

In [ ]:
provo = pd.read_csv('provo.csv')
provo.rename(columns = {'IA_DWELL_TIME':'time', 'Participant_ID': 'WorkerId', 'Word':'word', 
                        "Text_ID":"text_id", "Sentence_Number":"sentence_num"}, inplace = True)
provo = provo.dropna(subset=["Word_Number"])
provo = provo.astype({"Word_Number": 'Int64', "sentence_num": 'Int64'})
# First word isn't in dataset... Probably leads to artifically high surprisal
provo.loc[provo.Word_Number== 1]

In [ ]:
inds, paragraphs = zip(*provo[['text_id','Word_Number','word']].drop_duplicates().dropna().groupby(by = ['text_id']).apply(lambda x: ordered_string_join(zip(x['Word_Number'], x['word']), ' ')))
provo_stats = corpus_stats(enumerate(paragraphs,1), "provo_gpt_probs.pkl")

In [ ]:
provo['new_ind'] = provo.apply(lambda x: inds[x['text_id']-1].index(x["Word_Number"]), axis=1)
provo['sentence_num'] = provo.apply(lambda x: bisect.bisect(provo_stats['sent_markers'][x['text_id']], x['new_ind']), axis=1)
#sanity check
provo['ref_token'] = provo.apply(lambda x: provo_stats['split_string'][x['text_id']][x['new_ind']], axis=1) 
add_df_columns(provo, provo_stats, rolling_vals=True)

In [ ]:
provo_agg_per_sentence = produce_aggregate_per_sentence(provo, provo_stats)

### UCL Reading

In [ ]:
ucl = pd.read_csv('ucl/selfpacedreading.RT.txt','\t')
ucl.rename(columns = {'RT':'time', 'subj_nr': 'WorkerId', 
                        "sent_nr":"text_id"}, inplace = True)

In [ ]:
inds, paragraphs = zip(*ucl[['text_id','word_pos','word']].drop_duplicates().dropna().groupby(by = ['text_id']).apply(lambda x: ordered_string_join(zip(x['word_pos'], x['word']), ' ')))
ucl_stats = corpus_stats(enumerate(paragraphs,1), "ucl_gpt_probs.pkl")

In [ ]:
ucl['new_ind'] = ucl.apply(lambda x: inds[x['text_id']-1].index(x["word_pos"]), axis=1)
# ref token is sanity check. should be same as word
ucl['ref_token'] = ucl.apply(lambda x: ucl_stats['split_string'][x['text_id']][x['new_ind']], axis=1)
ucl['sentence_num'] = 0
add_df_columns(ucl, ucl_stats)

In [ ]:
ucl_agg_per_sentence = produce_aggregate_per_sentence(ucl, ucl_stats)

### UCL Eye

In [ ]:
ucl_eye = pd.read_csv('ucl/eyetracking.RT.txt','\t')
ucl_eye.rename(columns = {'RTfirstpass':'time', 'subj_nr': 'WorkerId', 
                        "sent_nr":"text_id"}, inplace = True)

In [ ]:
joined = ucl_eye[['text_id','word_pos','word']].drop_duplicates().dropna().groupby(by = ['text_id']).apply(lambda x: ordered_string_join(zip(x['word_pos'], x['word']), ' '))
inds, paragraphs = zip(*joined)
ucl_eye_stats = corpus_stats(zip(joined.index, paragraphs), "ucl_eye_gpt_probs.pkl")

In [ ]:
inds_dict = {i: ind_set for i, ind_set in zip(joined.index, inds)}
ucl_eye['new_ind'] = ucl_eye.apply(lambda x: inds_dict[x['text_id']].index(x["word_pos"]), axis=1)
ucl_eye['sentence_num'] = 0
ucl_eye['ref_token'] = ucl_eye.apply(lambda x: ucl_eye_stats['split_string'][x['text_id']][x['new_ind']], axis=1)
add_df_columns(ucl_eye, ucl_eye_stats)

In [ ]:
ucl_eye_agg_per_sentence = produce_aggregate_per_sentence(ucl_eye, ucl_eye_stats)

### CoLA

In [ ]:
cola = pd.read_csv('cola_public/raw/in_domain_train.tsv','\t', header=None, names=['ID','accept','NA','sentence'])
cola = cola.drop(columns='NA')
cola['text_id_'] = cola.index
cola['sentence_num_'] = 0
cola_stats = corpus_stats(enumerate(cola['sentence']), "cola_gpt_probs.pkl", split_sens=False)

In [ ]:
add_aggregate_cols(cola, cola_stats)

### EN Wiki

In [ ]:
enwiki = pd.read_csv('enwiki.csv','\t')
enwiki.rename(columns = {'mean_rating':'accept'}, inplace = True)
enwiki['text_id_'] = enwiki.index
enwiki['sentence_num_'] = 0
enwiki_stats = corpus_stats(enumerate(enwiki['text']), "enwiki_gpt_probs.pkl", split_sens=False)

In [ ]:
add_aggregate_cols(enwiki, enwiki_stats)

### Dundee Corpus

In [ ]:
def predict_encoding(file_path, n_lines=50):
    '''Predict a file's encoding using chardet'''
    import chardet

    # Open the file as binary data
    with open(file_path, 'rb') as f:
        # Join binary lines for specified number of lines
        rawdata = b''.join([f.readline() for _ in range(n_lines)])

    return chardet.detect(rawdata)['encoding']

In [ ]:
# s\w\d+ma2: data in fixation order
# WNUM in eyetracking DF maps to word index in text DF
dundeeDir = 'dundee/eye-tracking'
fileList = [os.path.join(dundeeDir, f) for f in os.listdir(dundeeDir) if re.match(r's\w\d+ma2p*\.dat', f)]
cols = ['WorkerId', 'text_id', 'WORD','TEXT','LINE','OLEN','WLEN','XPOS','WNUM','FDUR','OBLP','WDLP','FXNO','TXFR']
dundee = pd.DataFrame(columns = cols)
for file in fileList:
    temp = pd.read_csv(file, sep='\s+', encoding='Windows-1252')
    match = re.search(r'(s\w)(\d+)ma2p*\.dat', file.split('/')[-1])
    subjId = match.group(1)
    text = int(match.group(2))
    temp.insert(loc=0, column='text_id', value=text)
    temp.insert(loc=0, column='WorkerId', value=subjId)
    dundee = dundee.append(temp)
dundee.rename(columns = {'FDUR':'time', 'WORD':'word', 'WNUM': 'Word_Number'}, inplace = True)
dundee['time'] = dundee.time.astype('int64')
dundee = dundee.reset_index().drop(columns=['index','OLEN','XPOS','OBLP','WDLP','FXNO','TXFR'])

In [ ]:
dundeeDir = 'dundee/texts'
textList = [os.path.join(dundeeDir, f) for f in os.listdir(dundeeDir) if re.match(r'tx\d+wrdp\.dat', f)]
cols = ['word', 'text_id', 'screen_nr', 'line_nr', 'pos_on_line', 'serial_nr', 'initial_letter_position', 'word_len_punct', 'word_len', 'punc_code', 'n_chars_before','n_chars_after', 'Word_Number', 'local_word_freq']
dundeeTexts = pd.DataFrame(columns = cols)
for text in textList:
    temp = pd.read_csv(text, sep='\s+', names=cols, encoding='Windows-1252')
    dundeeTexts = dundeeTexts.append(temp)

In [ ]:
inds, paragraphs = zip(*dundeeTexts[['text_id','Word_Number','word']].drop_duplicates().dropna().groupby(by = ['text_id']).apply(lambda x: ordered_string_join(zip(x['Word_Number'], x['word']), ' ')))
dundee_stats = corpus_stats(enumerate(paragraphs,1), "dundee_gpt_probs.pkl")

In [ ]:
dundee['new_ind'] = dundee.apply(lambda x: inds[x['text_id']-1].index(x["Word_Number"]), axis=1)
dundee['sentence_num'] = dundee.apply(lambda x: bisect.bisect(dundee_stats['sent_markers'][x['text_id']], x['new_ind']), axis=1)
dundee['ref_token'] = dundee.apply(lambda x: dundee_stats['split_string'][x['text_id']][x['new_ind']], axis=1) 
add_df_columns(dundee, dundee_stats, rolling_vals=True)

In [ ]:
dundee_agg_per_sentence = produce_aggregate_per_sentence(dundee, dundee_stats, line_col='LINE')

## Experiments

In [ ]:
%%R
library(lme4)
library(MuMIn)
library(ggplot2)

In [ ]:
data = reading_times_df
aggregate_per_sentence = ns_agg_per_sentence

In [ ]:
data = provo
aggregate_per_sentence = provo_agg_per_sentence

In [ ]:
data = ucl
aggregate_per_sentence = ucl_agg_per_sentence

In [ ]:
data = ucl_eye
aggregate_per_sentence = ucl_eye_agg_per_sentence

In [ ]:
data = dundee
aggregate_per_sentence = dundee_agg_per_sentence

### Case Study 1

#### Psychometric Predictions

In [ ]:
%R -i aggregate_per_sentence

In [ ]:
%%R
lme_cross_val <- function(formula, df, d_var, num_folds=10, shuffle=FALSE){
    if(shuffle){
        df <- df[sample(nrow(df)),]
    }
    folds <- cut(seq(1,nrow(df)),breaks=num_folds,labels=FALSE)
    estimates <- c()
    for(i in 1:num_folds){
        testIndexes <- which(folds==i,arr.ind=TRUE)
        testData <- df[testIndexes,]
        trainData <- df[-testIndexes,]
        model <- lmer(formula, REML=FALSE, data=trainData)
        sigma <- mean(residuals(model)^2)
        estimate <- log(dnorm(testData[[d_var]], 
                              mean=predict(model, newdata=testData, allow.new.levels=TRUE), 
                              sd=sqrt(sigma)))
        estimates <- c(estimates, estimate)
    }
    estimates
}

In [ ]:
%%R
set.seed(42)
shuffled_order <- sample(nrow(aggregate_per_sentence))
baseline <- lme_cross_val("time_sum ~  time_count_nonzero  + (1 | WorkerId_)", 
                          aggregate_per_sentence[shuffled_order,],
                         'time_sum')

In [ ]:
%%R
powers_np_format <- c('1.0', '1.25', '1.5' , '1.75', '2.0', '2.25', '2.5' , '2.75', '3.0', '3.25', '3.5' , '3.75')
labels <- seq(1, 3.75, by=0.25)
agg_baseline <- lme_cross_val("time_sum ~  word_len_sum*freq_sum + time_count_nonzero + (1 | WorkerId_)", aggregate_per_sentence[shuffled_order,], 'time_sum')
power_func <- function(x){
    formula <- paste0("time_sum ~ log_prob_power_", x," :len + word_len_sum*freq_sum + time_count_nonzero  + (1 | WorkerId_)")
    cv <- lme_cross_val(formula,aggregate_per_sentence[shuffled_order,], 'time_sum')
    c(mean(cv-agg_baseline, na.rm=TRUE), var(cv-agg_baseline, na.rm=TRUE)/length(cv))
}
out <- cbind(labels, as.data.frame(do.call(rbind,lapply(powers_np_format, power_func))))

In [ ]:
%%R
ggplot(aes(x = labels, y = V1 ), data=out) + 
    geom_line() +
    geom_point(size=5) +
    geom_ribbon(aes(ymin=V1-sqrt(V2), ymax=V1+sqrt(V2)), alpha = 0.2, fill='red') +
    ylab("Per Sentence ∆LogLik") +
    xlab("k") +
    ggtitle("Corpus") +
    theme_minimal() +
    theme(text=element_text(size=25,family="serif"))
#ggsave('dundee.png', width = 8, height = 8, dpi=700)

#### Acceptability

In [ ]:
%R -i cola
%R -i enwiki

In [ ]:
%%R
d <- cola
family <- binomial

In [ ]:
%%R
d <- enwiki
family <- gaussian

In [ ]:
%%R
set.seed(42)
shuffled_order <- sample(nrow(d))
lm_cross_val <- function(formula, df, d_var, family, num_folds=10, shuffle=FALSE){
    if(shuffle){
        df <- df[sample(nrow(df)),]
    }
    folds <- cut(seq(1,nrow(df)),breaks=num_folds,labels=FALSE)
    estimates <- c()
    for(i in 1:num_folds){
        testIndexes <- which(folds==i,arr.ind=TRUE)
        testData <- df[testIndexes,]
        trainData <- df[-testIndexes,]
        model <- glm(formula, data=trainData, family=family)
        sigma <- mean(residuals(model)^2)
        if(identical(binomial, family)){
            predictions <- predict(model, newdata=testData, type="response")
        }else{
            predictions <- predict(model, newdata=testData)
        }
        estimate <- log(dnorm(testData[[d_var]], 
                              mean=predictions, 
                              sd=sqrt(sigma)))
        estimates <- c(estimates, estimate)
    }
    estimates
}

In [ ]:
%%R
powers_np_format <- c('1.0', '1.25', '1.5' , '1.75', '2.0', '2.25', '2.5' , '2.75', '3.0','3.25', '3.5' , '3.75')
powers <- seq(1, 3.75, by=0.25)
baseline <- lm_cross_val("accept ~  len ", 
                          d[shuffled_order,], 
                         'accept', 
                        family)
power_func <- function(x){
        name <- paste0("log_prob_power_",x,":len")
        formula <- paste0("accept ~ ", name)
        cv <- lm_cross_val(formula, d[shuffled_order,], 'accept', family)
        c(mean(cv-baseline, na.rm=TRUE), var(cv-baseline, na.rm=TRUE)/length(cv),mean(cv, na.rm=TRUE))
    }
out <- cbind(powers, as.data.frame(do.call(rbind,lapply(powers_np_format, power_func))))


In [ ]:
%%R
ggplot(aes(x = powers, y = V1 ), data=out) + 
    geom_line() +
    geom_point(size=5) +
    geom_ribbon(aes(ymin=V1-sqrt(V2), ymax=V1+sqrt(V2)), alpha = 0.2, fill='red') +
    ylab("Per Sentence ∆LogLik") +
    xlab("k") +
    ggtitle("Cola Corpus") +
    theme_minimal() +
    theme(text=element_text(size=25,family="serif"))
#ggsave('cola_pred_len_bs.png', width = 8, height = 8, dpi=700)

### Case Study 2

In [ ]:
%R -i aggregate_per_sentence

In [ ]:
attributes = ['mean','max','variance', 'variance_lang', 'ldiff', 'ldiff2', 'std']
for i in POWER_RANGE:
    attributes.extend(['entropy_'+str(i), 'entropy_n_'+str(i), 'power_'+str(i)])
attributes = sorted(attributes)
%R -i attributes

In [ ]:
%%R
set.seed(42)
shuffled_order <- sample(nrow(aggregate_per_sentence))
baseline <- lme_cross_val("time_sum ~   time_count_nonzero  + (1 | WorkerId_)", 
                          aggregate_per_sentence[shuffled_order,],
                         'time_sum')
for(var in attributes){
    predictor <- paste0("log_prob_",var,":len + time_count_nonzero")
    formula <- paste0("time_sum ~ ", predictor," + (1 | WorkerId_)")
    cat( mean(lme_cross_val(formula, aggregate_per_sentence[shuffled_order,], 'time_sum') - baseline, na.rm=TRUE)[[1]])
    cat('\n')

}
print('-------')
##  baseline with perplexity predictor (log probs are in nats)
baseline <- lme_cross_val("time_sum ~   I(exp(log_prob_mean)) + time_count_nonzero  + (1 | WorkerId_)", 
                          aggregate_per_sentence[shuffled_order,],
                         'time_sum')
for(var in attributes){
    predictor <- paste0("log_prob_",var, ":len + I(exp(log_prob_mean)) + time_count_nonzero")
    formula <- paste0("time_sum ~ ", predictor," + (1 | WorkerId_)")
    tryCatch({
        cat( mean(lme_cross_val(formula, aggregate_per_sentence[shuffled_order,], 'time_sum') - baseline, na.rm=TRUE)[[1]])
        cat('\n')
        }, error = function(error_condition) {
            print('-')
        })
    

}

#print(r.squaredGLMM(model)[1])

In [ ]:
%%R
baseline <- lm_cross_val("accept ~ len", 
                          d[shuffled_order,], 
                         'accept', 
                        family)

for(var in attributes){
    predictor <- paste0("len:log_prob_",var)
    formula <- paste0("accept ~ ", predictor)
    cat( mean(lm_cross_val(formula, d[shuffled_order,], 'accept', family) - baseline, na.rm=TRUE)[[1]])
    cat('\n')
    
}
print("-----")
##  baseline with perplexity predictor (log probs are in nats)
baseline <- lm_cross_val("accept ~ I(exp(log_prob_mean)) + len", 
                          d[shuffled_order,], 
                         'accept', 
                        family)
for(var in attributes){
    predictor <- paste0("I(exp(log_prob_mean)) + len:log_prob_",var)
    formula <- paste0("accept ~ ", predictor)
    cat( mean(lm_cross_val(formula, d[shuffled_order,], 'accept', family) - baseline, na.rm=TRUE)[[1]])
    cat('\n')
    
}


### Case Study 3

In [ ]:
# Need to do this each time you switch data sets!
%R -i data

In [ ]:
%%R 
set.seed(42)
shuffled_order <- sample(nrow(data))
powers <- seq(1.0, 2.75, by=0.25)
baseline <- lme_cross_val("time ~ freq*word_len + (1 | WorkerId)", data[shuffled_order,], 'time')

In [ ]:
%%R
out <- list()
names <- c('log_prob', 'prev_log_prob', 'prev2_log_prob','prev3_log_prob')
for(var in names){
    other_vars <- paste(setdiff(names, var), collapse=" + ")
    cur_baseline <- lme_cross_val(paste0("time ~ ", other_vars, " + freq*word_len + (1 | WorkerId)"), data[shuffled_order,], 'time')
    power_func <- function(x){
        data$log_prob_pow <- data[[var]]**x
        formula <- paste0("time ~ log_prob_pow +", other_vars, " + freq*word_len  + (1 | WorkerId)")
        cv <- lme_cross_val(formula, data[shuffled_order,], 'time')
        diff <- cv-baseline
        c(mean(diff, na.rm=TRUE), var(diff, na.rm=TRUE)/length(cv), mean(cv, na.rm=TRUE))
    }
    a <- do.call(rbind, lapply(powers, power_func))
    out[[var]] <- cbind(powers, as.data.frame(a))
}
power_func <- function(x){
        names <- c('log_prob', 'prev_log_prob', 'prev2_log_prob','prev3_log_prob')
        for(var in names){
            data[[paste0(var,'_pow')]] <- data[[var]]**x
        }
        formula <- paste0("time ~ log_prob_pow + prev_log_prob_pow + prev2_log_prob_pow +prev3_log_prob_pow + freq*word_len  + (1 | WorkerId)")
        cv <- lme_cross_val(formula, data[shuffled_order,], 'time')
        diff <- cv-baseline
        c(mean(diff, na.rm=TRUE), var(diff, na.rm=TRUE)/length(cv), mean(cv, na.rm=TRUE))
    }
a <- do.call(rbind, lapply(powers, power_func))
out[['all_k']] <- cbind(powers, as.data.frame(a))

In [ ]:
%%R
var1<-'log_prob'
var2<-'prev_log_prob'
var3<-'prev2_log_prob'
var4<-'prev3_log_prob'
ggplot(aes(x = powers, y = V1),data=out[[var1]]) + 
    geom_line(aes(color='0')) +
    geom_line(aes(color='-1'),data=out[[var2]]) +
    geom_line(aes(color='-2'),data=out[[var3]]) +
    geom_line(aes(color='-3'),data=out[[var4]]) +
    geom_point(size=2) +
    geom_point(data=out[[var2]], size=2) +
    geom_point(data=out[[var3]], size=2) +
    geom_point(data=out[[var4]], size=2) +
    aes(ymin=V1-sqrt(V2), ymax=V1+sqrt(V2)) +
    geom_ribbon(alpha = 0.2, fill='red') +
    geom_ribbon(data=out[[var2]], alpha = 0.2, fill='purple') +
    geom_ribbon(data=out[[var3]], alpha = 0.2, fill='blue') +
    geom_ribbon(data=out[[var4]], alpha = 0.2, fill='darkgreen') +
    scale_color_manual(name="Surprisal of Word at pos", values= c("0" = "red", "-1" = "purple", "-2" = "blue", "-3" = "darkgreen")) + 
    ylab("Per Token LogLik") +
    xlab("k") +
    ggtitle("Provo Corpus (with linear log-p predictors)") +
    theme_minimal()

## Misc

### Correlation

In [ ]:
names = ['log_prob_power_' + str(i) for i in POWER_RANGE]
r = enwiki.corr().accept[names]
r_se = np.sqrt((1-r**2)/(len(enwiki) - 2))
%R -i r
%R -i r_se
%R -i names
%R -i power_range

In [ ]:
%%R
library(ppcor)
p_r <- c()
p_r_se <- c()
for(i in names){
    test <- pcor.test(cola[['accept']], cola[[i]], cola[['len']])
    p_r <- c(p_r, test$estimate)
    p_r_se <- c(p_r_se, test$estimate/test$statistic)
}

In [ ]:
%%R
corrs <- as.data.frame(cbind(r=r,r_se=r_se,power_range))
library(ggplot2)
ggplot(aes(x = power_range, y = r ), data=corrs) + 
    geom_line() +
    geom_point(size=5) +
    geom_ribbon(aes(ymin=r-r_se, ymax=r+r_se), alpha = 0.2, fill='red') +
    theme_minimal() +
    labs(x = "k", y="Pearson's correlation coefficient", title="CoLA")+
    theme(text=element_text(size=25,family="serif")) +
    scale_y_reverse()
#ggsave('cola.png', width = 8, height = 8, dpi=700)

## Additional Plots

In [ ]:
%%R
library(tidyr)
centered_df <- pivot_longer(data, c('log_prob','prev_log_prob','prev2_log_prob','prev3_log_prob'), names_to="pos", values_to="log_prob")
ggplot(aes(x=log_prob, y=time, color=pos), data=centered_df) + 
    geom_smooth() +
    labs(title="Corpus")

In [ ]:
import plotnine
from itertools import cycle

In [ ]:
cola['accept'] = cola['accept'].astype('category')plotnine.ggplot() + \
    plotnine.aes(x='log_prob_mean', y="..density..", fill='accept') + \
    plotnine.geom_histogram(data=cola[cola['accept']=='1'], fill='blue', alpha = 0.5) +\
    plotnine.geom_histogram(data=cola[cola['accept']=='0'], fill='red', alpha = 0.5) +\
    plotnine.ylab("Density") + \
    plotnine.xlab("Mean Sentence Surprisal") + \
    plotnine.labels.ggtitle("CoLA corpus: blue = accept; red = reject") + \
    plotnine.scale_fill_manual(values = {'acceptable':'blue', 'not acceptable': 'red'})
